# DSPM Subnetting ExposureGraph — Lifecycle (v5.0.0)

Colab-native: no editable installs, no `python -m` module discovery.

In [22]:
# Setup G-Drive / mount & validate mount
import os
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Check if drive is already mounted
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print("Drive is already mounted.")

print("Done!")

# Check if any matching search_item was found before proceeding
search_item = "/content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/requirements.txt"
#search_item = "requirements.txt"
#!find /content/drive/MyDrive -type d -name "$search_item"
!find /content/drive/MyDrive -type f -wholename "$search_item"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive is already mounted.
Done!
/content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/requirements.txt


## CREATE

In [23]:
# Install runtime dependencies (pandas, networkx, etc.)

!pip -q install -r '/content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/requirements.txt'


In [24]:
# CREATE: build synthetic inventory + exposure graph + scoring artifacts
# Generate Synthetic Data


import json, os
from pathlib import Path
import pandas as pd
import networkx as nx
import random

def main():
    output_dir = Path("/content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/out")
    output_dir.mkdir(exist_ok=True)
    print(f"CREATE: building synthetic data in {output_dir}")

    # Generate synthetic assets with scores
    num_assets = 10
    assets_data = []
    for i in range(num_assets):
        asset_id = f"asset-{i:03d}"
        asset_name = f"server-{random.randint(100, 999)}.example.com"
        asset_type = random.choice(["VM", "Container", "Database", "Storage"])
        asset_score = random.uniform(0.1, 0.9)
        assets_data.append({
            "asset_id": asset_id,
            "name": asset_name,
            "type": asset_type,
            "score": asset_score
        })
    scores_df = pd.DataFrame(assets_data)
    scores_df.to_csv(output_dir / "scores_assets.csv", index=False)

    # Generate a simple synthetic exposure graph
    G = nx.erdos_renyi_graph(num_assets, 0.3, seed=42)
    # Map node indices to asset_ids
    mapping = {i: f"asset-{i:03d}" for i in range(num_assets)}
    G = nx.relabel_nodes(G, mapping)

    exposure_graph_data = nx.node_link_data(G)
    with open(output_dir / "exposure_graph.json","w") as f:
        json.dump(exposure_graph_data, f, indent=2)

    print("CREATE complete with comprehensive synthetic data")

if __name__ == "__main__":
    main()

CREATE: building synthetic data in /content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/out
CREATE complete with comprehensive synthetic data


## MAINTAIN

In [25]:
# MAINTAIN: re-run scoring after a simulated change (idempotent)

import pandas as pd
display(pd.read_csv("/content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/out/scores_assets.csv"))


,asset_id,name,type,score
0,asset-000,server-632.example.com,Database,0.352008
1,asset-001,server-909.example.com,Container,0.425245
2,asset-002,server-575.example.com,Database,0.813192
3,asset-003,server-232.example.com,VM,0.861452
4,asset-004,server-143.example.com,Storage,0.283213
5,asset-005,server-977.example.com,VM,0.869122
6,asset-006,server-917.example.com,Database,0.881891
7,asset-007,server-684.example.com,VM,0.781947
8,asset-008,server-888.example.com,Storage,0.773592
9,asset-009,server-691.example.com,Container,0.842205


## AUDIT

In [26]:
# AUDIT: evidence bundle (artifacts you can attach to a ticket)
from pathlib import Path
import hashlib

out = Path("out")
artifacts = sorted([p for p in out.glob("*") if p.is_file()])
print("Artifacts:", [p.name for p in artifacts])

def sha256(p: Path) -> str:
    h = hashlib.sha256()
    h.update(p.read_bytes())
    return h.hexdigest()

for p in artifacts:
    print(p.name, sha256(p)[:16], "...")  # short hash for readability


Artifacts: ['exposure_graph.json', 'scores_assets.csv']
exposure_graph.json db48ec76b25560dc ...
scores_assets.csv fc18fd2f4444506d ...


In [27]:
import os
import pandas as pd
import json

print("Contents of 'out' directory:", os.listdir('out'))

# Display scores_assets.csv
print("\nContent of scores_assets.csv:")
display(pd.read_csv('out/scores_assets.csv'))

# Display exposure_graph.json
print("\nContent of exposure_graph.json:")
with open('out/exposure_graph.json', 'r') as f:
    exposure_graph_data = json.load(f)
    print(json.dumps(exposure_graph_data, indent=2))

Contents of 'out' directory: ['exposure_graph.json', 'scores_assets.csv']

Content of scores_assets.csv:


,asset_id,name,type,score
0,asset-000,server-954.example.com,VM,0.711461
1,asset-001,server-848.example.com,Database,0.890270
2,asset-002,server-614.example.com,Database,0.796871
3,asset-003,server-306.example.com,Container,0.381422
4,asset-004,server-449.example.com,Database,0.358210
5,asset-005,server-126.example.com,Database,0.751405
6,asset-006,server-458.example.com,Container,0.470153
7,asset-007,server-656.example.com,Storage,0.474956
8,asset-008,server-212.example.com,Container,0.658025
9,asset-009,server-841.example.com,Container,0.397929



Content of exposure_graph.json:
{
  "directed": false,
  "multigraph": false,
  "graph": {},
  "nodes": [
    {
      "id": "asset-000"
    },
    {
      "id": "asset-001"
    },
    {
      "id": "asset-002"
    },
    {
      "id": "asset-003"
    },
    {
      "id": "asset-004"
    },
    {
      "id": "asset-005"
    },
    {
      "id": "asset-006"
    },
    {
      "id": "asset-007"
    },
    {
      "id": "asset-008"
    },
    {
      "id": "asset-009"
    }
  ],
  "edges": [
    {
      "source": "asset-000",
      "target": "asset-002"
    },
    {
      "source": "asset-000",
      "target": "asset-003"
    },
    {
      "source": "asset-000",
      "target": "asset-004"
    },
    {
      "source": "asset-000",
      "target": "asset-008"
    },
    {
      "source": "asset-001",
      "target": "asset-002"
    },
    {
      "source": "asset-001",
      "target": "asset-003"
    },
    {
      "source": "asset-001",
      "target": "asset-005"
    },
    {
      "sou

## DESTROY

In [28]:
# ARCHIVE: copy generated artifacts to a timestamped folder and write a receipt
import os, json
from pathlib import Path
from datetime import datetime
import shutil # Import shutil for copying files

base_output_dir = Path("/content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/out")

# Create a timestamped archive directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
archive_dir = base_output_dir / f"archive_{timestamp}"
archive_dir.mkdir(parents=True, exist_ok=True)

print(f"Archiving artifacts from {base_output_dir} to {archive_dir}")

archived_files = []
for p in base_output_dir.glob("*"):
    # Only copy files, and exclude any archive directories already present
    if p.is_file() and not p.name.startswith("archive_"):
        try:
            new_path = archive_dir / p.name
            shutil.copy2(p, new_path) # Copy the file
            archived_files.append(str(new_path))
        except OSError as e:
            print(f"Error archiving {p}: {e}")

# Write an archive receipt into the archive directory
receipt_path = archive_dir / "archive_receipt.json"

receipt_content = {
    "archived_files": archived_files,
    "archive_timestamp": timestamp,
    "status": "artifacts_archived"
}
with open(receipt_path, "w") as f:
    json.dump(receipt_content, f, indent=2)

print(f"Archive receipt written to {receipt_path.resolve()}")
print(f"Contents of {archive_dir}: {os.listdir(archive_dir)}")
print(f"Contents of {base_output_dir} (after archiving): {os.listdir(base_output_dir)}")
with open(receipt_path) as f:
    loaded_receipt = json.load(f)
    print("archive_receipt.json content:", loaded_receipt,"\n\n\n")
    # pretty print
    print("archive_receipt.json content (pretty):", json.dumps(loaded_receipt, indent=2))

Archiving artifacts from /content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/out to /content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/out/archive_20260210_132534
Archive receipt written to /content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/out/archive_20260210_132534/archive_receipt.json
Contents of /content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/out/archive_20260210_132534: ['scores_assets.csv', 'exposure_graph.json', 'archive_receipt.json']
Contents of /content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/out (after archiving): ['scores_assets.csv', 'exposure_graph.json', 'archive_20260210_132430', 'archive_20260210_132534']
archive_receipt.json content: {'archived_files': ['/content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/out/archive_20260210_132534/scores_assets.csv', '/content/drive/MyDrive/DSPM-Subnetting-ExposureGraph-Agnostic/out/archive_20260210_132534/exposure_graph.json'], 'archive_timestamp': '20260210_1325